In [1]:
import ipyleaflet
import ipywidgets
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, Point

In [2]:
path_inventario = '../LHC_Hidroweb/invent2.csv'

In [3]:
df = pd.read_csv(path_inventario, engine='python', sep='\t', delimiter=';')

In [4]:
df.columns

Index(['RegistroID', 'Importado', 'Temporario', 'Removido',
       'ImportadoRepetido', 'BaciaCodigo', 'SubBaciaCodigo', 'RioCodigo',
       'EstadoCodigo', 'MunicipioCodigo', 'ResponsavelCodigo',
       'ResponsavelUnidade', 'ResponsavelJurisdicao', 'OperadoraCodigo',
       'OperadoraUnidade', 'OperadoraSubUnidade', 'TipoEstacao', 'Codigo',
       'Nome', 'CodigoAdicional', 'Latitude', 'Longitude', 'Altitude',
       'AreaDrenagem', 'TipoEstacaoEscala', 'TipoEstacaoRegistradorNivel',
       'TipoEstacaoDescLiquida', 'TipoEstacaoSedimentos',
       'TipoEstacaoQualAgua', 'TipoEstacaoPluviometro',
       'TipoEstacaoRegistradorChuva', 'TipoEstacaoTanqueEvapo',
       'TipoEstacaoClimatologica', 'TipoEstacaoPiezometria',
       'TipoEstacaoTelemetrica', 'PeriodoEscalaInicio', 'PeriodoEscalaFim',
       'PeriodoRegistradorNivelInicio', 'PeriodoRegistradorNivelFim',
       'PeriodoDescLiquidaInicio', 'PeriodoDescLiquidaFim',
       'PeriodoSedimentosInicio', 'PeriodoSedimentosFim',
      

In [6]:
df[['Latitude', 'Longitude']] = df[['Latitude', 'Longitude']].apply(lambda x: x.str.replace(',','.'))
df['Latitude'] = df['Latitude'].astype('float')
df['Longitude'] = df['Longitude'].astype('float')

In [7]:
df[['Latitude','Longitude']].head()

,Latitude,Longitude
0,-13.6500,-73.0833
1,-13.0167,-73.9833
2,-13.6500,-73.3667
3,-13.4333,-73.8333
4,-13.6500,-73.9500


In [8]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude,df.Latitude))

In [10]:
m01 = ipyleaflet.Map(zoom=2)

# Layers
## All points for HEATMAP
heatmap_all = ipyleaflet.Heatmap(locations=[tuple(r) for r in df[['Latitude', 'Longitude']].to_numpy()],
                                 radius=30, name='All point Heatmap')
m01.add_layer(heatmap_all)
radius_slider = ipywidgets.IntSlider(description='Radius', min=1, max=50, value=15)
ipywidgets.jslink((radius_slider,'value'),(heatmap_all,'radius'))
widget_control01 = ipyleaflet.WidgetControl(widget=radius_slider, position='topright')
m01.add_control(widget_control01)

# Controles
## Layer
control_layer = ipyleaflet.LayersControl(position='topright')
m01.add_control(control_layer)

## Fullscreen
control_fullscreen = ipyleaflet.FullScreenControl()
m01.add_control(control_fullscreen)

## Desenho
control_desenho = ipyleaflet.DrawControl()
feature_collection = {'type': 'FeatureCollection',
                      'features': []}
def handle_draw(self, action, geo_json):
    """Do something with the GeoJSON when it's drawn on the map"""   
    feature_collection['features'].append(geo_json)

control_desenho.on_draw(handle_draw)
m01.add_control(control_desenho)

## Mediçao
# control_medicao = ipyleaflet.MeasureControl(position='bottomleft',
#                                             active_color='orange',
#                                             primary_length_units='kilometers')
# m01.add_control(control_medicao)
# control_medicao.complete_color = 'red'

## Escala
control_escala = ipyleaflet.ScaleControl(position='bottomleft')
m01.add_control(control_escala)

m01

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…